In [1]:
import docx2txt
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np   
from bs4 import BeautifulSoup
from pathlib import Path
import os
import re
import requests
import sys
import csv
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
text_pattern = re.compile(r'\w+')
# extract text
text = docx2txt.process("ASU_2014_09 details.docx")
word_list = text_pattern.findall(text)

str_ = ' '.join(word_list)

In [3]:
str_list = str_.split('Step')
len(str_list)

8

In [4]:
for i in range(len(str_list)):
    print(str_list[i][:20], i)

 0
 1 Identifying the C 1
 2 Identifying Perfo 2
 5 Satisfaction of P 3
 5 1 Performance Obl 4
 5 2 Performance Obl 5
 3 Determining the T 6
 4 Allocating the Tr 7


In [5]:
step_list = []
step_list.append(re.sub(r'[^A-Za-z ]+', '', str_list[1]).lower())
step_list.append(re.sub(r'[^A-Za-z ]+', '',str_list[2]).lower())
step_list.append(re.sub(r'[^A-Za-z ]+', '',str_list[6]).lower())
step_list.append(re.sub(r'[^A-Za-z ]+', '',str_list[7]).lower())
step_list.append(re.sub(r'[^A-Za-z ]+', '',str_list[3] + ' ' + str_list[4] + ' ' + str_list[5]).lower())
step_list.append(re.sub(r'[^A-Za-z ]+', '',str_list[4]).lower())
step_list.append(re.sub(r'[^A-Za-z ]+', '',str_list[5]).lower())
step_list.append(re.sub(r'[^A-Za-z ]+', '',str_).lower())
len(step_list)

8

In [6]:
cachedStopWords = stopwords.words("english")
def stopFunc(text):
    
    text = ' '.join([word for word in text.split() if word not in cachedStopWords])
    text = ' '.join([word for word in text.split() if len(word)>1])
    return text
#text = 'hello bye the The hi or an Or An it It' 
#text = stopFunc(text)
#text

In [12]:
ASU_list = []
for txt in step_list:
    ASU_list.append(stopFunc(txt))


In [13]:
ASU_list[1]

'identifying performance obligations contract inception entity shall assess goods services promised contract customer shall identify performance obligation promise transfer customer either good service bundle goods services distinct series distinct goods services substantially pattern transfer customer see paragraph series distinct goods services pattern transfer customer following criteria met distinct good service series entity promises transfer customer would meet criteria paragraph performance obligation satisfied time accordance paragraphs method would used measure entity progress toward complete satisfaction performance obligation transfer distinct good service series customer promises contracts customers contract customer generally explicitly states goods services entity promises transfer customer however performance obligations identified contract customer may limited goods services explicitly stated contract contract customer also may include promises implied entity customary 

In [14]:
#Get count vectors for 1-gram in ASU document
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(ASU_list)
header = vectorizer.get_feature_names()
count_vectors = X.toarray()
my_df = pd.DataFrame(count_vectors)

my_df.to_csv('counts_ASU.csv', index=False, header=header)

In [27]:
#Get count vectors for 2-gram and 3-gram in ASU document
vectorizer = CountVectorizer(ngram_range = (2,3))
X = vectorizer.fit_transform(ASU_list)
header = vectorizer.get_feature_names()
count_vectors = X.toarray()
my_df = pd.DataFrame(count_vectors)

my_df.to_csv('counts_ASU_ngram.csv', index=False, header=header)

Now moving to Sales Contracts

In [16]:
with open('sales_contracts.csv') as f:
    sales = [s for line in f.readlines() for s in line[:-1].split(',')]
sales = sales[1:]    
sales

['https://www.sec.gov/Archives/edgar/data/23194/000095013404011584/d16950exv10w3.htm',
 'https://www.sec.gov/Archives/edgar/data/23217/000162828017007184/cag-5x28x2017x10kex10321.htm',
 'https://www.sec.gov/Archives/edgar/data/29002/000119312513327122/d579876dex102.htm',
 'https://www.sec.gov/Archives/edgar/data/29332/000002933219000067/a102psafor3201susan.htm',
 'https://www.sec.gov/Archives/edgar/data/31978/000119312504191140/dex1002.htm ',
 'https://www.sec.gov/Archives/edgar/data/31978/000119312505099245/dex1003.htm ',
 'https://www.sec.gov/Archives/edgar/data/31978/000119312507109598/dex1004.htm ',
 'https://www.sec.gov/Archives/edgar/data/31978/000119312508226233/dex1006.htm ',
 'https://www.sec.gov/Archives/edgar/data/31978/000119312509101641/dex1003.htm ',
 'https://www.sec.gov/Archives/edgar/data/31978/000119312509167228/dex1006.htm ',
 'https://www.sec.gov/Archives/edgar/data/31978/000119312511049952/dex104203.htm ',
 'https://www.sec.gov/Archives/edgar/data/31978/00011931251

In [17]:
sales_list = []
k = 1
for clink in sales:
    try:
        print(k, clink)
        k+=1
        res = requests.get(clink)
        html_page = res.text
        soup = BeautifulSoup(html_page)
        raw_text = soup.body.get_text(" ",strip = True).replace("\n", " ").replace(u'\xa0', ' ').replace('-', ' ')
        raw_text = ' '.join(raw_text.split())
        raw_text = re.sub(r'[^A-Za-z ]+', '', raw_text).lower()
        sales_list.append(stopFunc(raw_text))
    except Exception as e:
        print(e)
        sales_list.append('xxxxxxx')
        
        
    
   

1 https://www.sec.gov/Archives/edgar/data/23194/000095013404011584/d16950exv10w3.htm
2 https://www.sec.gov/Archives/edgar/data/23217/000162828017007184/cag-5x28x2017x10kex10321.htm
3 https://www.sec.gov/Archives/edgar/data/29002/000119312513327122/d579876dex102.htm
4 https://www.sec.gov/Archives/edgar/data/29332/000002933219000067/a102psafor3201susan.htm
5 https://www.sec.gov/Archives/edgar/data/31978/000119312504191140/dex1002.htm 
6 https://www.sec.gov/Archives/edgar/data/31978/000119312505099245/dex1003.htm 
7 https://www.sec.gov/Archives/edgar/data/31978/000119312507109598/dex1004.htm 
8 https://www.sec.gov/Archives/edgar/data/31978/000119312508226233/dex1006.htm 
9 https://www.sec.gov/Archives/edgar/data/31978/000119312509101641/dex1003.htm 
10 https://www.sec.gov/Archives/edgar/data/31978/000119312509167228/dex1006.htm 
11 https://www.sec.gov/Archives/edgar/data/31978/000119312511049952/dex104203.htm 
12 https://www.sec.gov/Archives/edgar/data/31978/000119312511212664/dex1004.htm

97 https://www.sec.gov/Archives/edgar/data/88205/000110465915067759/a15-20300_1ex10d1.htm
98 https://www.sec.gov/Archives/edgar/data/88790/000089710119000976/a192156_ex10-1.htm
99 https://www.sec.gov/Archives/edgar/data/89439/000119312513329383/d582826dex101.htm
100 https://www.sec.gov/Archives/edgar/data/93389/000090901208000440/exh10-22.txt
101 https://www.sec.gov/Archives/edgar/data/96699/000095012310047910/c00675exv10w1.htm
102 https://www.sec.gov/Archives/edgar/data/96699/000095012310047910/c00675exv10w2.htm
103 https://www.sec.gov/Archives/edgar/data/96699/000095012310047910/c00675exv10w3.htm
104 https://www.sec.gov/Archives/edgar/data/96699/000095012310115828/c10038exv10w14.htm
105 https://www.sec.gov/Archives/edgar/data/96699/000095012310115828/c10038exv10w15.htm
106 https://www.sec.gov/Archives/edgar/data/96699/000095012310115828/c10038exv10w16.htm
107 https://www.sec.gov/Archives/edgar/data/96699/000095012311048303/c16791exv10w1.htm
108 https://www.sec.gov/Archives/edgar/data

191 https://www.sec.gov/Archives/edgar/data/939767/000104746904005598/a2129329zex-10_41.htm
192 https://www.sec.gov/Archives/edgar/data/939767/000119312505100938/dex104.htm
193 https://www.sec.gov/Archives/edgar/data/939767/000119312505161077/dex104.htm
194 https://www.sec.gov/Archives/edgar/data/939767/000119312506049562/dex1036.htm
195 https://www.sec.gov/Archives/edgar/data/939767/000119312507173665/dex101.htm
196 https://www.sec.gov/Archives/edgar/data/939767/000119312507235074/dex101.htm
197 https://www.sec.gov/Archives/edgar/data/939767/000119312507235074/dex102.htm
198 https://www.sec.gov/Archives/edgar/data/939767/000119312507235074/dex103.htm
199 https://www.sec.gov/Archives/edgar/data/939767/000119312508104066/dex101.htm
200 https://www.sec.gov/Archives/edgar/data/939767/000119312509049879/dex1064.htm
201 https://www.sec.gov/Archives/edgar/data/939767/000119312509049879/dex1066.htm
202 https://www.sec.gov/Archives/edgar/data/939767/000119312509159532/dex104.htm
203 https://ww

283 https://www.sec.gov/Archives/edgar/data/946644/000149315217004866/ex10-1.htm
284 https://www.sec.gov/Archives/edgar/data/946644/000149315218004291/ex10-45.htm
285 https://www.sec.gov/Archives/edgar/data/946644/000149315218004291/ex10-46.htm
286 https://www.sec.gov/Archives/edgar/data/946644/000149315218004291/ex10-47.htm
287 https://www.sec.gov/Archives/edgar/data/946644/000149315218004291/ex10-48.htm
288 https://www.sec.gov/Archives/edgar/data/946644/000149315218016149/ex10-1.htm
289 https://www.sec.gov/Archives/edgar/data/946644/000149315220004768/ex10-1.htm
290 https://www.sec.gov/Archives/edgar/data/946644/000149315220005912/ex10-1.htm
291 https://www.sec.gov/Archives/edgar/data/946644/000149315220015932/ex10-1.htm
292 https://www.sec.gov/Archives/edgar/data/946644/000149315220015932/ex10-3.htm
293 https://www.sec.gov/Archives/edgar/data/946644/000149315220015932/ex10-4.htm
294 https://www.sec.gov/Archives/edgar/data/948708/000095012310019966/a55282exv10w4w3.htm
295 https://www

377 https://www.sec.gov/Archives/edgar/data/1001316/000114420414068736/v393077_ex10-1.htm
378 https://www.sec.gov/Archives/edgar/data/1001316/000114420415002790/v399000_ex10-1.htm
379 https://www.sec.gov/Archives/edgar/data/1001316/000165495418008750/exhibit10-20.htm
380 https://www.sec.gov/Archives/edgar/data/1001316/000165495419009176/tgtx_ex101.htm
381 https://www.sec.gov/Archives/edgar/data/1001385/000119312510093466/dex102.htm
382 https://www.sec.gov/Archives/edgar/data/1001385/000119312510170831/dex101.htm
383 https://www.sec.gov/Archives/edgar/data/1001385/000119312510179558/dex102.htm
384 https://www.sec.gov/Archives/edgar/data/1001385/000119312510223941/dex102.htm
385 https://www.sec.gov/Archives/edgar/data/1001385/000119312510238156/dex102.htm
386 https://www.sec.gov/Archives/edgar/data/1001385/000119312512438704/d430766dex102.htm
387 https://www.sec.gov/Archives/edgar/data/1005201/000104746914002523/a2218683zex-10_50.htm
388 https://www.sec.gov/Archives/edgar/data/1005201/00

...

2199 https://www.sec.gov/Archives/edgar/data/1446847/000155837017003785/irwd-20170331ex102005a26.htm
2200 https://www.sec.gov/Archives/edgar/data/1446847/000141057819000630/tv526520_ex10-1.htm
2201 https://www.sec.gov/Archives/edgar/data/1446847/000141057819001289/tv529547_ex10-1.htm
2202 https://www.sec.gov/Archives/edgar/data/1447028/000117184314001457/exh_1034.htm
2203 https://www.sec.gov/Archives/edgar/data/1447028/000117184315001398/exh_1041.htm
2204 https://www.sec.gov/Archives/edgar/data/1447028/000117184315002398/exh_1042.htm
2205 https://www.sec.gov/Archives/edgar/data/1447028/000162828017002760/exhibit1067alexionagreemen.htm
2206 https://www.sec.gov/Archives/edgar/data/1447028/000162828017005644/exhibit1067alexionagreemen.htm
2207 https://www.sec.gov/Archives/edgar/data/1447028/000144702820000118/a2020-q210xqex103cross.htm
2208 https://www.sec.gov/Archives/edgar/data/1447028/000144702820000118/a2020-q210xqex1041stam.htm
2209 https://www.sec.gov/Archives/edgar/data/1447028/000

2288 https://www.sec.gov/Archives/edgar/data/1487952/000120677410001567/exhibit10-13.htm
2289 https://www.sec.gov/Archives/edgar/data/1491576/000110465914033822/a14-9607_1ex10d1.htm
2290 https://www.sec.gov/Archives/edgar/data/1491576/000155837016007470/tsro-20160630ex101a241bc.htm
2291 https://www.sec.gov/Archives/edgar/data/1491576/000155837016009294/tsro-20160930ex101c18f00.htm
2292 https://www.sec.gov/Archives/edgar/data/1491576/000155837017001160/tsro-20161231ex1026faaa1.htm
2293 https://www.sec.gov/Archives/edgar/data/1491576/000155837017001998/tsro_ex101.htm
2294 https://www.sec.gov/Archives/edgar/data/1491576/000155837017008418/tsro-20170930ex1012e9301.htm
2295 https://www.sec.gov/Archives/edgar/data/1492426/000119312514096677/d681079dex1039.htm
2296 https://www.sec.gov/Archives/edgar/data/1492426/000119312515067257/d843160dex1077.htm
2297 https://www.sec.gov/Archives/edgar/data/1492426/000156459019016824/ex103.htm
2298 https://www.sec.gov/Archives/edgar/data/1492426/0001564590

2375 https://www.sec.gov/Archives/edgar/data/1534525/000168316818000831/xenetic_10k-ex1046.htm
2376 https://www.sec.gov/Archives/edgar/data/1535379/000162828018004005/q4-1810kexhibit1017.htm
2377 https://www.sec.gov/Archives/edgar/data/1535379/000162828018004005/q4-1810kexhibit1018.htm
2378 https://www.sec.gov/Archives/edgar/data/1535379/000162828018004005/q4-1810kexhibit1019.htm
2379 https://www.sec.gov/Archives/edgar/data/1535379/000162828018004005/q4-1810kexhibit1020.htm
2380 https://www.sec.gov/Archives/edgar/data/1537054/000119312514193970/d714152dex10150.htm
2381 https://www.sec.gov/Archives/edgar/data/1537054/000119312514193970/d714152dex10151.htm
2382 https://www.sec.gov/Archives/edgar/data/1537054/000119312514304568/d764488dex10152.htm
2383 https://www.sec.gov/Archives/edgar/data/1537054/000119312515280656/d73619dex10145.htm
2384 https://www.sec.gov/Archives/edgar/data/1537054/000119312515280656/d73619dex10146.htm
2385 https://www.sec.gov/Archives/edgar/data/1537054/0001193125

2464 https://www.sec.gov/Archives/edgar/data/1583648/000158364818000026/ex104.htm
2465 https://www.sec.gov/Archives/edgar/data/1583648/000158364818000032/a18050320181q10qexhibit101.htm
2466 https://www.sec.gov/Archives/edgar/data/1583648/000158364818000043/seagencollaborationagreeme.htm
2467 https://www.sec.gov/Archives/edgar/data/1583648/000158364820000046/a200602_pirs-sgen1stamen.htm
2468 https://www.sec.gov/Archives/edgar/data/1583648/000158364820000046/a200602pirssgenamendment.htm
2469 https://www.sec.gov/Archives/edgar/data/1585608/000104746917000684/a2230899zex-10_41.htm
2470 https://www.sec.gov/Archives/edgar/data/1585608/000104746917000684/a2230899zex-10_42.htm
2471 https://www.sec.gov/Archives/edgar/data/1585608/000110465920047211/a20-15990_1ex10d1.htm
2472 https://www.sec.gov/Archives/edgar/data/1586049/000143774915014658/ex10-1.htm
2473 https://www.sec.gov/Archives/edgar/data/1586049/000143774917003451/ex10-20.htm
2474 https://www.sec.gov/Archives/edgar/data/1586049/00014377

2553 https://www.sec.gov/Archives/edgar/data/1664703/000166470319000008/ex1030amend1to3rdarppavipu.htm
2554 https://www.sec.gov/Archives/edgar/data/1664703/000166470319000008/ex1031amend2to3rdarppavpum.htm
2555 https://www.sec.gov/Archives/edgar/data/1664703/000166470319000025/ex10-3fuelcellsystemsupply.htm
2556 https://www.sec.gov/Archives/edgar/data/1664703/000166470319000025/ex10-8purchaseuseandmainte.htm
2557 https://www.sec.gov/Archives/edgar/data/1664703/000166470320000013/ex1033bmccapexagreement.htm
2558 https://www.sec.gov/Archives/edgar/data/1664703/000166470320000057/ex102redactedarpuma.htm
2559 https://www.sec.gov/Archives/edgar/data/1664703/000166470320000061/ex102redactedarpuma.htm
2560 https://www.sec.gov/Archives/edgar/data/1667633/000156459019042650/ftsv-ex101_54.htm
2561 https://www.sec.gov/Archives/edgar/data/1671584/000156459016022097/apvo-ex106_60.htm
2562 https://www.sec.gov/Archives/edgar/data/1671584/000156459018005384/apvo-ex105_281.htm
2563 https://www.sec.gov/

In [22]:
#Getting count vectors for 1-gram in sales contracts eleminating words with frequency less than 0.02
vectorizer = CountVectorizer(min_df = 0.02)#, ngram_range = (2,3))
X = vectorizer.fit_transform(sales_list)
header = vectorizer.get_feature_names()
count_vectors = X.toarray()
my_df = pd.DataFrame(count_vectors)

my_df.to_csv('count_sales_lf.csv', index=False, header=header)

In [23]:
#Getting count vectors for 1-gram in sales contracts eleminating words with frequency less than 0.02 and more than 0.95
vectorizer = CountVectorizer(min_df = 0.02, max_df = 0.95)#, ngram_range = (2,3))
X = vectorizer.fit_transform(sales_list)
header = vectorizer.get_feature_names()
count_vectors = X.toarray()
my_df = pd.DataFrame(count_vectors)

my_df.to_csv('count_sales_hf.csv', index=False, header=header)

In [24]:
#Getting count vectors for 2-gram and 3-gram in sales contracts eleminating words with frequency less than 0.02 
vectorizer = CountVectorizer(min_df = 0.02, ngram_range = (2,3))
X = vectorizer.fit_transform(sales_list)
header = vectorizer.get_feature_names()
count_vectors = X.toarray()
my_df = pd.DataFrame(count_vectors)

my_df.to_csv('count_sales_lf_ngrams.csv', index=False, header=header)

In [25]:
#Getting count vectors for 2-gram and 3-gram in sales contracts eleminating words with frequency less than 0.02 and more than 0.95
vectorizer = CountVectorizer(min_df = 0.02, max_df = 0.95, ngram_range = (2,3))
X = vectorizer.fit_transform(sales_list)
header = vectorizer.get_feature_names()
count_vectors = X.toarray()
my_df = pd.DataFrame(count_vectors)

my_df.to_csv('count_sales_hf_ngrams.csv', index=False, header=header)

Moving on to calculate cosine similarity

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
#calculate cosine similarily:
def cos_sim(arr1, arr2):
    out_arr = cosine_similarity(arr1, arr2)
    return out_arr 

In [38]:
results = []
with open("count__ASU.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        results.append(row)
results = np.array([np.array(res) for res in results])
header = results[0,1:]
results = results[1:, 1:].astype(float)
counts = np.sum(results, axis = 0)
counts = np.reshape(counts, (1,6092))
with open("count_vectors_ASU.csv", 'a') as f:
    writer = csv.writer(f)
    writer.writerow(counts)
#my_df = pd.DataFrame(counts)
#my_df.to_csv('count_vectors_ngrams_doc.csv', index=False, header=header)

In [28]:
#Part 1 of 1-gram
df1 = pd.read_csv('counts_ASU.csv')
#print(df1.head)
df2 = pd.read_csv('count_sales_lf.csv')
#print(df2.head)
common_columns = df1.columns.union(df2.columns)

df1 = df1.reindex(columns=common_columns, fill_value='0')
df2 = df2.reindex(columns=common_columns, fill_value='0')

df_main = pd.concat([df1, df2], sort=False, ignore_index=True)
#print(df_main.head)
temp_arr = df_main.values.astype(float)

arr2 = temp_arr[:8,:]
arr1 = temp_arr[8:,:]

res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cosine_sim_lf.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6,7
0,0.041094,0.061822,0.045205,0.049736,0.039882,0.050034,0.020483,0.057998
1,0.103441,0.039311,0.092484,0.072212,0.083950,0.056135,0.068131,0.100423
2,0.173501,0.074985,0.151917,0.144997,0.161724,0.098173,0.164767,0.180328
3,0.198282,0.091046,0.171710,0.168598,0.183282,0.127261,0.167527,0.206754
4,0.016748,0.006927,0.010957,0.014316,0.000000,0.000000,0.000000,0.012298


In [30]:
#Part 2 of 1-gram
df1 = pd.read_csv('counts_ASU.csv')
#print(df1.head)
df2 = pd.read_csv('count_sales_hf.csv')
#print(df2.head)
common_columns = df1.columns.union(df2.columns)

df1 = df1.reindex(columns=common_columns, fill_value='0')
df2 = df2.reindex(columns=common_columns, fill_value='0')

df_main = pd.concat([df1, df2], sort=False, ignore_index=True)
#print(df_main.head)
temp_arr = df_main.values.astype(float)

arr2 = temp_arr[:8,:]
arr1 = temp_arr[8:,:]

res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cosine_sim_hf.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6,7
0,0.041094,0.061822,0.045205,0.049736,0.039882,0.050034,0.020483,0.057998
1,0.103441,0.039311,0.092484,0.072212,0.083950,0.056135,0.068131,0.100423
2,0.173501,0.074985,0.151917,0.144997,0.161724,0.098173,0.164767,0.180328
3,0.198282,0.091046,0.171710,0.168598,0.183282,0.127261,0.167527,0.206754
4,0.016748,0.006927,0.010957,0.014316,0.000000,0.000000,0.000000,0.012298


In [40]:
results = []
with open("counta_ASU_ngram.csv") as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        results.append(row)
results = np.array([np.array(res) for res in results])
header = results[0,1:]
results = results[1:, 1:].astype(float)
counts = np.sum(results, axis = 0)
counts = np.reshape(counts, (1,6092))
with open("count_vectors_ASU_ngram.csv", 'a') as f:
    writer = csv.writer(f)
    writer.writerow(counts)

array([['Step', 'ability direct', 'ability direct use',
        'ability intention', 'ability intention pay'],
       ['1', '0', '0', '1', '1'],
       ['2', '0', '0', '0', '0'],
       ['3', '0', '0', '0', '0'],
       ['4', '0', '0', '0', '0']], dtype='<U40')

In [32]:
#part 1 of n-gram
df1 = pd.read_csv('counts_ASU_ngram.csv')
#print(df1.head)
df2 = pd.read_csv('count_sales_lf_ngrams.csv')
#print(df2.head)
common_columns = df1.columns.union(df2.columns)

df1 = df1.reindex(columns=common_columns, fill_value='0')
df2 = df2.reindex(columns=common_columns, fill_value='0')

df_main = pd.concat([df1, df2], sort=False, ignore_index=True)
#print(df_main.head)
temp_arr = df_main.values.astype(float)

arr2 = temp_arr[:8,:]
arr1 = temp_arr[8:,:]

res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cosine_sim_lf_ngrams.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000738,0.000000,0.000000,0.000000,0.000000,0.000000,0.000182
1,0.000000,0.000000,0.000984,0.000000,0.000000,0.000000,0.000000,0.000361
2,0.006105,0.000000,0.006147,0.000541,0.001298,0.000000,0.001644,0.004439
3,0.011928,0.002185,0.002790,0.006155,0.007652,0.002208,0.002737,0.009377
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
#part 2 of n-gram
df1 = pd.read_csv('counts_ASU_ngram.csv')
#print(df1.head)
df2 = pd.read_csv('count_sales_hf_ngrams.csv')
#print(df2.head)
common_columns = df1.columns.union(df2.columns)

df1 = df1.reindex(columns=common_columns, fill_value='0')
df2 = df2.reindex(columns=common_columns, fill_value='0')

df_main = pd.concat([df1, df2], sort=False, ignore_index=True)
#print(df_main.head)
temp_arr = df_main.values.astype(float)

arr2 = temp_arr[:8,:]
arr1 = temp_arr[8:,:]

res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cosine_sim_hf_ngrams.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000738,0.000000,0.000000,0.000000,0.000000,0.000000,0.000182
1,0.000000,0.000000,0.000984,0.000000,0.000000,0.000000,0.000000,0.000361
2,0.006105,0.000000,0.006147,0.000541,0.001298,0.000000,0.001644,0.004439
3,0.011928,0.002185,0.002790,0.006155,0.007652,0.002208,0.002737,0.009377
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Implementing tf-idf

In [35]:
#Part 3 of 1-gram
vectorizer = TfidfVectorizer(min_df = 0.02)
X = vectorizer.fit_transform(ASU_list + sales_list)
#print(vectorizer.get_feature_names())
feature_names = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df.head()

,aa,aaa,aab,ab,abandon,abandoned,abandonment,abbreviated,abide,ability,...,yearly,years,yes,yet,yield,york,zealand,zero,zip,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016140,...,0.0,0.0,0.0,0.092975,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.012262,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041486,...,0.0,0.0,0.0,0.011380,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
temp_arr = df.values.astype(float)
arr2 = temp_arr[:8,:]#.reshape((1,temp_arr.shape[1]))
arr1 = temp_arr[8:,:]
res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cosine_sim_tfidf.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6,7
0,0.020360,0.028977,0.021314,0.021512,0.016245,0.015042,0.007578,0.026665
1,0.050994,0.018838,0.041772,0.032711,0.032890,0.021265,0.025610,0.045659
2,0.097041,0.046361,0.077857,0.079011,0.075578,0.041210,0.075881,0.096150
3,0.104341,0.048680,0.085175,0.084682,0.079950,0.049721,0.072076,0.103238
4,0.020192,0.003972,0.012117,0.008154,0.000000,0.000000,0.000000,0.011081


In [37]:
#Part 3 of 2-gram and 3 grams
vectorizer = TfidfVectorizer(min_df = 0.02, ngram_range = (2,3))
X = vectorizer.fit_transform(ASU_list + sales_list)
#print(vectorizer.get_feature_names())
feature_names = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df.head()

,abandoned finally,ability defend,ability grant,ability others,ability others access,ability perform,ability perform obligations,absolute discretion,abuse act,abuse act national,...,york new york,york ny,york united,york united states,york usa,york without,york without giving,york without regard,zip code,zip code amendment
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
temp_arr = df.values.astype(float)
arr2 = temp_arr[:8,:]#.reshape((1,temp_arr.shape[1]))
arr1 = temp_arr[8:,:]
res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cosine_sim_tfidf_ngrams.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000153
1,0.000000,0.000000,0.000330,0.000000,0.000000,0.000000,0.000000,0.000092
2,0.004299,0.000000,0.004649,0.000478,0.000804,0.000000,0.000995,0.002593
3,0.009239,0.002451,0.004325,0.007169,0.016043,0.006677,0.002660,0.009914
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [43]:
results = results[1:, 1:].astype(float)

In [44]:
results.shape


(7, 6092)

In [46]:
counts = np.sum(results, axis = 0)
counts = np.reshape(counts, (1,6092))
counts.shape

(1, 6092)

In [47]:
my_df = pd.DataFrame(counts)

my_df.to_csv('count_vectors_ngrams_doc.csv', index=False, header=header)

In [48]:
results

array([[0., 0., 1., ..., 2., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [6., 6., 0., ..., 0., 0., 0.],
       [5., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
df1 = pd.read_csv('count_vectors.csv')
#print(df1.head)
df2 = pd.read_csv('count_vectors_sales_lf.csv')
#print(df2.head)
common_columns = df1.columns.union(df2.columns)

df1 = df1.reindex(columns=common_columns, fill_value='0')
df2 = df2.reindex(columns=common_columns, fill_value='0')

df_main = pd.concat([df1, df2], sort=False, ignore_index=True)
#print(df_main.head)
temp_arr = df_main.values.astype(float)

arr2 = temp_arr[:8,:]
arr1 = temp_arr[8:,:]

res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cos_sim_lf_all.csv', index=False)
my_df.head()

In [55]:
# for part 1 in 1-gram

df1 = pd.read_csv('count_vectors_ASU_ngrams.csv')
print(df1.head())

df2 = pd.read_csv('count_vectors_sales_lf_ngrams.csv')
print(df2.head())


#my_df.to_csv('count_vectors_sales_mf.csv', index=False, header=header)


   ability  able  accept  acceptance  accepted  access  accordance  \
0     14.0   7.0     1.0         4.0       2.0     4.0        35.0   

   accordingly  account  accounted  ...  willing  within  without  words  \
0          1.0     16.0        6.0  ...      1.0     5.0      2.0    1.0   

   work  would  writing  written  year  yet  
0   1.0   24.0      2.0      1.0   3.0  9.0  

[1 rows x 733 columns]
   aa  aaa  aab  ab  abandon  abandoned  abandonment  abbreviated  abide  \
0   0    0    0   0        0          0            0            0      0   
1   0    0    0   0        0          0            0            0      0   
2   0    0    0   0        0          0            0            0      0   
3   0    0    0   0        0          0            0            0      0   
4   0    0    0   0        0          0            0            0      0   

   ability  ...  year  yearly  years  yes  yet  yield  york  zero  zip  zone  
0        0  ...     0       0      0    0    0      0 

In [56]:
common_columns = df1.columns.union(df2.columns)

df1 = df1.reindex(columns=common_columns, fill_value='0')
df2 = df2.reindex(columns=common_columns, fill_value='0')

df_main = pd.concat([df1, df2], sort=False, ignore_index=True)
df_main.head()

,aa,aaa,aab,ab,abandon,abandoned,abandonment,abbreviated,abide,ability,...,year,yearly,years,yes,yet,yield,york,zero,zip,zone
0,0,0,0,0,0,0,0,0,0,14.0,...,3.0,0,0,0,9.0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0.0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1.0,...,0.0,0,0,0,0.0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0.0,...,0.0,0,1,0,0.0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,2.0,...,10.0,0,8,0,2.0,0,0,0,0,12


In [58]:
temp_arr = df_main.values.astype(float)
temp_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [59]:
arr2 = temp_arr[0,:].reshape((1,temp_arr.shape[1]))
arr1 = temp_arr[1:,:]
print(arr1.shape, arr2.shape)

(2601, 5191) (1, 5191)


In [61]:
res_arr = cos_sim(arr1, arr2)
res_arr.shape

(2601, 1)

In [64]:
my_df = pd.DataFrame(res_arr)

my_df.to_csv('cos_sim_lf.csv', index=False, header = ['cos_sim_lf'])
my_df.head()

,0
0,0.238700
1,0.100985
2,0.184854
3,0.211326
4,0.058190


In [78]:
df1 = pd.read_csv('count_vectors.csv')
#print(df1.head)
df2 = pd.read_csv('count_vectors_sales_lf.csv')
#print(df2.head)
common_columns = df1.columns.union(df2.columns)

df1 = df1.reindex(columns=common_columns, fill_value='0')
df2 = df2.reindex(columns=common_columns, fill_value='0')

df_main = pd.concat([df1, df2], sort=False, ignore_index=True)
#print(df_main.head)
temp_arr = df_main.values.astype(float)

arr2 = temp_arr[:8,:]
arr1 = temp_arr[8:,:]

res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cos_sim_lf_all.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6
0,0.238086,0.193491,0.195446,0.171880,0.168835,0.121437,0.155772
1,0.103433,0.039308,0.092477,0.072206,0.083944,0.056130,0.068126
2,0.173506,0.074987,0.151922,0.145001,0.161728,0.098176,0.164772
3,0.198281,0.091046,0.171708,0.168597,0.183280,0.127260,0.167525
4,0.041094,0.061822,0.045205,0.049736,0.039882,0.050034,0.020483


In [75]:
my_df.to_csv('cos_sim_lf_ngrams_all.csv', index=False)

In [3]:
#TF-IDF
text_pattern = re.compile(r'\w+')
# extract text
text = docx2txt.process("ASU_2014_09 details.docx")
word_list = text_pattern.findall(text)

str_ = ' '.join(word_list)
str_ = re.sub(r'[^A-Za-z ]+', '', str_).lower()
str_ = stopFunc(str_)

str_

        
        
    
   

'step identifying contract entity shall account contract customer within scope topic following criteria met parties contract approved contract writing orally accordance customary business practices committed perform respective obligations entity identify party rights regarding goods services transferred entity identify payment terms goods services transferred contract commercial substance risk timing amount entity future cash flows expected change result contract probable entity collect consideration entitled exchange goods services transferred customer evaluating whether collectibility amount consideration probable entity shall consider customer ability intention pay amount consideration due amount consideration entity entitled may less price stated contract consideration variable entity may offer customer price concession see paragraph contract agreement two parties creates enforceable rights obligations enforceability rights obligations contract matter law contracts written oral imp

In [36]:
with open('sales_contracts.csv') as f:
    sales = [s for line in f.readlines() for s in line[:-1].split(',')]
sales = sales[1:]    



#stop_list.append(str_)
k = 1

for clink in sales:
    try:
        print(k, clink, stop_list[k-1][:20])
        k+=1
        res = requests.get(clink)
        html_page = res.text
        soup = BeautifulSoup(html_page)
        # title = soup.find('title')
        # title_text = title.text if title else 'No Title'
        raw_text = soup.body.get_text(" ",strip = True).replace("\n", " ").replace(u'\xa0', ' ').replace('-', ' ')
        raw_text = ' '.join(raw_text.split())
        raw_text = re.sub(r'[^A-Za-z ]+', '', raw_text).lower()
        stop_list.append(stopFunc(raw_text))
    except Exception as e:
        print(e)
        stop_list.append('Nothing here to see')

1 https://www.sec.gov/Archives/edgar/data/23194/000095013404011584/d16950exv10w3.htm identifying contract
2 https://www.sec.gov/Archives/edgar/data/23217/000162828017007184/cag-5x28x2017x10kex10321.htm identifying performa
3 https://www.sec.gov/Archives/edgar/data/29002/000119312513327122/d579876dex102.htm determining transact
4 https://www.sec.gov/Archives/edgar/data/29332/000002933219000067/a102psafor3201susan.htm allocating transacti
5 https://www.sec.gov/Archives/edgar/data/31978/000119312504191140/dex1002.htm  satisfaction perform
6 https://www.sec.gov/Archives/edgar/data/31978/000119312505099245/dex1003.htm  performance obligati
7 https://www.sec.gov/Archives/edgar/data/31978/000119312507109598/dex1004.htm  performance obligati
8 https://www.sec.gov/Archives/edgar/data/31978/000119312508226233/dex1006.htm  ex dexvwhtm services
9 https://www.sec.gov/Archives/edgar/data/31978/000119312509101641/dex1003.htm  ex cag xxxkexhtm exh
10 https://www.sec.gov/Archives/edgar/data/31978/00011

80 https://www.sec.gov/Archives/edgar/data/69733/000143774917018197/ex_97755.htm us securities exchan
81 https://www.sec.gov/Archives/edgar/data/69733/000143774917018197/ex_97756.htm us securities exchan
82 https://www.sec.gov/Archives/edgar/data/70415/000114420413057906/v357900_ex10-1.htm ex ex htm exhibit ex
83 https://www.sec.gov/Archives/edgar/data/70415/000007041519000050/exhibit1039123118.htm ex ex ahtm ex exhibi
84 https://www.sec.gov/Archives/edgar/data/70415/000007041519000065/exhibit103912311810-ka.htm ex ex htm ex exhibit
85 https://www.sec.gov/Archives/edgar/data/74046/000120677407002369/exhibit10-2.htm ex ex ahtm ex exhibi
86 https://www.sec.gov/Archives/edgar/data/74046/000114420407066302/v096599_ex10-1.htm ex ex htm exhibit ex
87 https://www.sec.gov/Archives/edgar/data/74046/000007404616000094/odcex10407312016.htm ex exhtm exhibit ima
88 https://www.sec.gov/Archives/edgar/data/74046/000007404620000083/exhibit101odcamproduct.htm us securities exchan
89 https://www.sec.gov

156 https://www.sec.gov/Archives/edgar/data/927003/000095012311078905/c21578exv10w1.htm ex xexhtm ex ex exhi
157 https://www.sec.gov/Archives/edgar/data/927003/000095012311093790/c22970exv10w1.htm ex vrml xexhtm ex ex
158 https://www.sec.gov/Archives/edgar/data/927003/000095012311093787/c23798exv10w1.htm us securities exchan
159 https://www.sec.gov/Archives/edgar/data/928054/000092805419000033/exhibit102-supplyagree.htm ex dexvwhtm global s
160 https://www.sec.gov/Archives/edgar/data/928340/000095014408002040/g12298exv10w27w2.htm ex dexvwhtm shipping
161 https://www.sec.gov/Archives/edgar/data/928340/000095014408002040/g12298exv10w27w3.htm ex dexvwhtm ex exvw 
162 https://www.sec.gov/Archives/edgar/data/928340/000114420411006075/v210070_ex10-1.htm ex dexvwhtm ex exvw 
163 https://www.sec.gov/Archives/edgar/data/929008/000095015206010447/l23890aexv10w1.htm ex cexvwhtm exhibit 
164 https://www.sec.gov/Archives/edgar/data/929008/000095015209001958/l35104aexv10w46.htm ex cexvwhtm exhibit 


233 https://www.sec.gov/Archives/edgar/data/944480/000094448006000067/license_agreement.htm ex exelexhibithtm ex
234 https://www.sec.gov/Archives/edgar/data/944809/000095014407002945/g06337exv10w12.htm ex exelexhibithtm ex
235 https://www.sec.gov/Archives/edgar/data/944809/000095012311026008/g26471exv10w27.htm ex exelexhibithtm ex
236 https://www.sec.gov/Archives/edgar/data/944809/000095012311069279/g27424exv10w27.htm ex exelexhibithtm ex
237 https://www.sec.gov/Archives/edgar/data/944809/000094480915000003/opk-12312014xex1023.htm us securities exchan
238 https://www.sec.gov/Archives/edgar/data/944809/000094480916000025/opk-6302016xex102.htm ex exelexhibithtm ex
239 https://www.sec.gov/Archives/edgar/data/944809/000094480918000014/ex1026-opkoxjtdevlicenseag.htm ex contracthtm esa c
240 https://www.sec.gov/Archives/edgar/data/944809/000094480920000033/a101viforlicagmtamend.htm ex licenseagreementh
241 https://www.sec.gov/Archives/edgar/data/944809/000094480920000033/a102pfizerlicagmtame

309 https://www.sec.gov/Archives/edgar/data/1000623/000110465905009819/a05-3116_1ex10d12d7.htm ex yexvwhtm ex distr
310 https://www.sec.gov/Archives/edgar/data/1000623/000110465905009819/a05-3116_1ex10d12d8.htm ex exhibithtm exhibi
311 https://www.sec.gov/Archives/edgar/data/1000623/000110465906014372/a06-5855_1ex10d12d9.htm ex exhibithtm exhibi
312 https://www.sec.gov/Archives/edgar/data/1000623/000110465906014372/a06-5855_1ex10d12d10.htm ex exhibithtm exhibi
313 https://www.sec.gov/Archives/edgar/data/1000623/000095012309030368/c88460exv10w12w11.htm ex exhibithtm exhibi
314 https://www.sec.gov/Archives/edgar/data/1000623/000114420410041236/v192088_ex10-12x3.htm us securities exchan
315 https://www.sec.gov/Archives/edgar/data/1000694/000095013301502210/w52015ex10-3.htm us securities exchan
316 https://www.sec.gov/Archives/edgar/data/1000694/000095013301502210/w52015ex10-4.htm us securities exchan
317 https://www.sec.gov/Archives/edgar/data/1000694/000095013309001586/w73981exv10w5.htm 

...

2254 https://www.sec.gov/Archives/edgar/data/1474439/000147443916000162/tnavex101634331201610q.htm ex tnavexkhtm amendm
2255 https://www.sec.gov/Archives/edgar/data/1474439/000147443917000009/tnavex1016381231201610q.htm ex tnavexkhtm amendm
2256 https://www.sec.gov/Archives/edgar/data/1474439/000147443917000057/tnav9301710qex101639.htm ex tnavexqhtm fourth
2257 https://www.sec.gov/Archives/edgar/data/1474439/000147443917000057/tnav09301710-qex102621.htm ex tnavexqhtm amendm
2258 https://www.sec.gov/Archives/edgar/data/1474439/000147443918000011/tnav12311710-qex102623.htm us securities exchan
2259 https://www.sec.gov/Archives/edgar/data/1474439/000147443918000025/tnavex102624331201810q.htm ex tnavex qhtm amend
2260 https://www.sec.gov/Archives/edgar/data/1474439/000147443918000025/tnavex102625331201810q.htm ex tnavexkhtm exhibi
2261 https://www.sec.gov/Archives/edgar/data/1474439/000147443918000057/tnavex1016469301810q.htm ex tnavexqhtm territ
2262 https://www.sec.gov/Archives/edgar/dat

2326 https://www.sec.gov/Archives/edgar/data/1515673/000156459018002535/rare-ex1019_867.htm ex exportfolioagreem
2327 https://www.sec.gov/Archives/edgar/data/1515673/000156459020004667/rare-ex1025_647.htm ex ddexhtm ex ex exh
2328 https://www.sec.gov/Archives/edgar/data/1524358/000119312515064836/d843780dex1030.htm ex vex htm oem purch
2329 https://www.sec.gov/Archives/edgar/data/1524358/000119312515064836/d843780dex1031.htm us securities exchan
2330 https://www.sec.gov/Archives/edgar/data/1524358/000119312515064836/d843780dex1032.htm ex vex htm master se
2331 https://www.sec.gov/Archives/edgar/data/1526119/000155837018006754/vstm-20180630ex101492686.htm us securities exchan
2332 https://www.sec.gov/Archives/edgar/data/1526119/000155837018008809/vstm-20180930ex101157ce4.htm ex rare exhtm ex rar
2333 https://www.sec.gov/Archives/edgar/data/1526119/000155837019009444/vstm-20190930ex1018e984c.htm ex rare exhtm ex rar
2334 https://www.sec.gov/Archives/edgar/data/1528837/000155837017004047/

2398 https://www.sec.gov/Archives/edgar/data/1537054/000119312519046812/d680726dex10152.htm ex ddexhtm ex ex exh
2399 https://www.sec.gov/Archives/edgar/data/1537054/000119312519046812/d680726dex10153.htm ex ddexhtm ex ex exh
2400 https://www.sec.gov/Archives/edgar/data/1537054/000119312520072797/d857662dex10141.htm ex ddexhtm ex ex exh
2401 https://www.sec.gov/Archives/edgar/data/1537054/000119312520138765/d897159dex10143.htm ex ddexhtm ex ex exh
2402 https://www.sec.gov/Archives/edgar/data/1537054/000119312520138765/d897159dex10144.htm us securities exchan
2403 https://www.sec.gov/Archives/edgar/data/1537054/000119312520166064/d857100dex101.htm ex ddexhtm ex ex exh
2404 https://www.sec.gov/Archives/edgar/data/1537054/000156459020038939/gogo-ex10146_184.htm ex ddexhtm ex ex exh
2405 https://www.sec.gov/Archives/edgar/data/1537435/000153743519000006/billingagreementrecogenone.htm ex ddexhtm ex ex exh
2406 https://www.sec.gov/Archives/edgar/data/1538217/000149315220020418/ex10-02.htm ex

2470 https://www.sec.gov/Archives/edgar/data/1585608/000104746917000684/a2230899zex-10_42.htm us securities exchan
2471 https://www.sec.gov/Archives/edgar/data/1585608/000110465920047211/a20-15990_1ex10d1.htm ex exhtm exhibit exh
2472 https://www.sec.gov/Archives/edgar/data/1586049/000143774915014658/ex10-1.htm ex aqqexhibithtm exh
2473 https://www.sec.gov/Archives/edgar/data/1586049/000143774917003451/ex10-20.htm ex seagencollaborati
2474 https://www.sec.gov/Archives/edgar/data/1586049/000143774918003397/ex_104989.htm ex apirs sgenstamenh
2475 https://www.sec.gov/Archives/edgar/data/1586049/000143774918014062/ex_118360.htm ex apirssgenamendmen
2476 https://www.sec.gov/Archives/edgar/data/1586049/000143774918020161/ex_127311.htm ex azex htm ex exhib
2477 https://www.sec.gov/Archives/edgar/data/1586049/000143774919015566/ex_151472.htm us securities exchan
2478 https://www.sec.gov/Archives/edgar/data/1588972/000156459020008538/reph-ex1029_299.htm ex exdhtm ex exhibit
2479 https://www.sec

2545 https://www.sec.gov/Archives/edgar/data/1654151/000165415120000006/dcph-20200331xex102.htm ex ex htm exhibit ex
2546 https://www.sec.gov/Archives/edgar/data/1658566/000165856618000056/exhibit101purchaseandsalea.htm ex ex htm exhibit ex
2547 https://www.sec.gov/Archives/edgar/data/1658566/000165856618000069/exhibit101.htm ex ex dhtm ex editex
2548 https://www.sec.gov/Archives/edgar/data/1658566/000119312520047447/d877498dex101.htm ex ex dhtm ex editex
2549 https://www.sec.gov/Archives/edgar/data/1658566/000165856620000074/exhibit101bppsaamendme.htm us securities exchan
2550 https://www.sec.gov/Archives/edgar/data/1662524/000156459018027289/akca-ex101_90.htm ex exhibithtm exhibi
2551 https://www.sec.gov/Archives/edgar/data/1662524/000156459020008119/akca-ex1015_357.htm ex ddexhtm ex ex exh
2552 https://www.sec.gov/Archives/edgar/data/1664703/000166470319000008/ex10293rdarpurchaseusean.htm us securities exchan
2553 https://www.sec.gov/Archives/edgar/data/1664703/000166470319000008/ex

In [39]:

vectorizer = TfidfVectorizer(min_df = 0.02)
X = vectorizer.fit_transform(stop_list)
#print(vectorizer.get_feature_names())
feature_names = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df.head()

,aa,aaa,aab,ab,abandon,abandoned,abandonment,abbreviated,abide,ability,...,yearly,years,yes,yet,yield,york,yr,zero,zip,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015843,...,0.0,0.0,0.0,0.093614,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.012454,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040674,...,0.0,0.0,0.0,0.011445,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
temp_arr = df.values.astype(float)
arr2 = temp_arr[:7,:]#.reshape((1,temp_arr.shape[1]))
arr1 = temp_arr[7:,:]
res_arr = cos_sim(arr1, arr2)

my_df = pd.DataFrame(res_arr)

my_df.to_csv('cos_sim_tfidf_all.csv', index=False)
my_df.head()

,0,1,2,3,4,5,6
0,0.161532,0.149095,0.122640,0.111384,0.095640,0.058753,0.089921
1,0.050241,0.018699,0.040909,0.032488,0.032288,0.020936,0.025097
2,0.096415,0.046311,0.076956,0.079381,0.075295,0.041060,0.075692
3,0.020148,0.028801,0.021035,0.021583,0.016034,0.014791,0.007571
4,0.020025,0.004051,0.011974,0.008373,0.000000,0.000000,0.000000


In [22]:
arr1.shape

(2601, 5073)

In [23]:
arr2.shape

(5073,)

In [42]:
import pickle
with open("all_non_stop_words.txt", "wb") as fp:   #Pickling
    pickle.dump(stop_list, fp)

In [35]:
len(stop_list)

7